In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
import re
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , confusion_matrix, classification_report

In [3]:
Train_Tweet = pd.read_csv('train.csv')
Train_Tweet.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
print(Train_Tweet.shape)

(7613, 5)


In [5]:
X1 = Train_Tweet.drop('target',axis=1)
Y1 = Train_Tweet['target']

In [6]:
messages1 = X1.copy()
corpus1 = []
for i in range(0,len(messages1)):
    X1 = re.sub(r'https?:\/\/.*[\r\n]*', '', messages1['text'][i])
    X1 = re.sub(r'[^a-zA-Z]', ' ', X1)
    X1= re.sub(r'#', '', X1)
    corpus1.append(X1)
print(corpus1)

['Our Deeds are the Reason of this  earthquake May ALLAH Forgive us all', 'Forest fire near La Ronge Sask  Canada', 'All residents asked to  shelter in place  are being notified by officers  No other evacuation or shelter in place orders are expected', '       people receive  wildfires evacuation orders in California ', 'Just got sent this photo from Ruby  Alaska as smoke from  wildfires pours into a school ', ' RockyFire Update    California Hwy     closed in both directions due to Lake County fire    CAfire  wildfires', ' flood  disaster Heavy rain causes flash flooding of streets in Manitou  Colorado Springs areas', 'I m on top of the hill and I can see a fire in the woods   ', 'There s an emergency evacuation happening now in the building across the street', 'I m afraid that the tornado is coming to our area   ', 'Three people died from the heat wave so far', 'Haha South Tampa is getting flooded hah  WAIT A SECOND I LIVE IN SOUTH TAMPA WHAT AM I GONNA DO WHAT AM I GONNA DO FVCK  fl

In [7]:
voc_size = 50000
onehot_repr1 = [one_hot(words,voc_size) for words in list(corpus1)]
print(onehot_repr1)

[[24710, 15649, 8945, 36382, 43124, 14853, 20588, 2204, 32235, 33274, 44573, 27114, 22465], [36917, 44061, 4772, 43308, 10153, 44149, 41322], [22465, 29162, 1725, 40944, 30645, 29076, 26089, 8945, 3337, 49942, 33121, 16019, 38407, 38389, 12049, 25713, 30645, 29076, 26089, 47900, 8945, 26304], [8576, 41155, 3739, 12049, 47900, 29076, 2714], [36681, 27245, 44917, 20588, 5984, 36720, 33625, 44633, 18943, 3242, 36720, 3739, 10261, 13860, 30774, 18129], [3743, 47054, 2714, 45159, 4537, 29076, 43792, 31012, 13936, 40944, 8297, 34804, 44061, 30918, 3739], [28697, 34676, 8597, 4582, 43044, 18603, 16114, 14853, 3612, 29076, 22479, 38007, 19187, 4545], [46359, 11231, 30709, 40922, 14853, 36382, 10573, 1244, 46359, 3619, 40531, 30774, 44061, 29076, 36382, 48160], [48465, 4190, 19551, 33509, 12049, 44653, 32205, 29076, 36382, 30295, 39491, 36382, 30470], [46359, 11231, 26849, 32480, 36382, 16113, 5813, 43539, 40944, 24710, 29342], [35809, 8576, 48320, 36720, 36382, 45318, 14701, 33666, 14262], [38

In [8]:
Length_of_tweet=20
embedded_docs1 = pad_sequences(np.array(onehot_repr1),padding='pre',maxlen=Length_of_tweet)
print(embedded_docs1)

[[    0     0     0 ... 44573 27114 22465]
 [    0     0     0 ... 10153 44149 41322]
 [ 1725 40944 30645 ... 47900  8945 26304]
 ...
 [    0     0     0 ... 14853 17962 41904]
 [15448 19551 35843 ... 16113 42236 12799]
 [    0     0     0 ...  1161 36179 46782]]


In [9]:
feature_vector = 256
model=Sequential()
model.add(Embedding(voc_size,feature_vector,input_length=Length_of_tweet))
model.add(LSTM(256, return_sequences = True))
model.add(Dropout(0.4))
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 256)           12800000  
_________________________________________________________________
lstm (LSTM)                  (None, 20, 256)           525312    
_________________________________________________________________
dropout (Dropout)            (None, 20, 256)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               197120    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 13,522,561
Trainable params: 13,522,561
Non-trainable params: 0
____________________________________________

In [10]:
len(embedded_docs1),Y1.shape

(7613, (7613,))

In [11]:
X1 = np.array(embedded_docs1)
Y1 = np.array(Y1)
X1.shape,Y1.shape

((7613, 20), (7613,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.33, random_state=42)

In [13]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=256)

Epoch 1/30
20/20 [==============================] - 25s 1s/step - loss: 0.6605 - accuracy: 0.6127 - val_loss: 0.5696 - val_accuracy: 0.7461
Epoch 2/30
20/20 [==============================] - 20s 990ms/step - loss: 0.4238 - accuracy: 0.8214 - val_loss: 0.4566 - val_accuracy: 0.7990
Epoch 3/30
20/20 [==============================] - 25s 1s/step - loss: 0.2454 - accuracy: 0.9043 - val_loss: 0.5177 - val_accuracy: 0.7780
Epoch 4/30
20/20 [==============================] - 21s 1s/step - loss: 0.1563 - accuracy: 0.9475 - val_loss: 0.5890 - val_accuracy: 0.7720
Epoch 5/30
20/20 [==============================] - 20s 990ms/step - loss: 0.1129 - accuracy: 0.9633 - val_loss: 0.6710 - val_accuracy: 0.7485
Epoch 6/30
20/20 [==============================] - 19s 961ms/step - loss: 0.0903 - accuracy: 0.9671 - val_loss: 0.6932 - val_accuracy: 0.7553
Epoch 7/30
20/20 [==============================] - 20s 989ms/step - loss: 0.0793 - accuracy: 0.9727 - val_loss: 0.8046 - val_accuracy: 0.7473
Epoch 8/

In [14]:
y_pred = model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [15]:
confusion_matrix(y_pred,y_test)

array([[1147,  352],
       [ 299,  715]], dtype=int64)

In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.79      0.77      0.78      1499
           1       0.67      0.71      0.69      1014

    accuracy                           0.74      2513
   macro avg       0.73      0.74      0.73      2513
weighted avg       0.74      0.74      0.74      2513



In [17]:
accuracy_score(y_pred,y_test)

0.7409470752089137

In [18]:
model.save("Disaster_Model.h5")